In [ ]:
#import sys 
#sys.path.append('../../env/lib/python3.10/site-packages/') 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
import time
from datetime import timedelta
from tensorflow.keras.models import load_model,model_from_json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import optuna
from optuna.samplers import TPESampler
from pickle import dump,load
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np

#for lr in [1e-8,1e-7,1e-6]:
#    for bs in [4,8,16]:
#iris = sklearn.datasets.load_iris()
physical_devices=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
SEED = 7
tf.keras.utils.set_random_seed(SEED) #Establece la semilla de Python, Numpy y TensorFlow. Para que los números aleatorios sean los mismos cada vez que se corre
tf.config.experimental.enable_op_determinism() #Para que con los mismos inputs, se tengan los mismos outputs cada vez que se corre'''
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
img_size = 512
initial_learning_rate=1e-3
batch_size = 4
image_size = (img_size,img_size)

train_ds = tf.keras.utils.image_dataset_from_directory(
"../../datasets/dataset_5/training/",
seed=1337,
#color_mode='grayscale',
image_size=image_size,
batch_size=1)

test_ds = tf.keras.utils.image_dataset_from_directory(
"../../datasets/dataset_5/test/",
seed=1337, #1337
image_size=image_size,
#color_mode='grayscale',
#shuffle=False,
batch_size=batch_size)

kf = KFold(n_splits=5,shuffle=True)

def dataset_to_arrays(ds):
    list_ds = list(ds)
    images = []
    labels = []
    
    # Loop through the batches
    for i in range(len(list_ds)):
        img = list_ds[i][0][0].numpy()
        lab = list_ds[i][1][0].numpy()  # Extract the element and convert to a NumPy array
        labels.append(lab)  # Append the element to the list
        images.append(img)  # Append the element to the list
        
    return images, labels

trainX, trainY = dataset_to_arrays(train_ds)

trainX = np.array(trainX)
trainY = np.array(trainY)

for train_index, val_index in kf.split(trainX):
    print("TRAIN: ", len(train_index), "VAL: ", len(val_index))
    trainX_k, valX_k = trainX[train_index], trainX[val_index]
    trainY_k, valY_k= trainY[train_index], trainY[val_index]

    ##################### Pre-trained model #####################
            
    model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_size,img_size,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
    ])

    # Create tensor flow datasets to apply prefetch
    '''train_ds = tf.data.Dataset.from_tensor_slices((trainX_k, trainY_k))
    val_ds = tf.data.Dataset.from_tensor_slices((valX_k, valY_k))

    # Shuffle and batch dataset
    #SHUFFLE_BUFFER_SIZE = 1378
    #train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
    train_ds = train_ds.batch(batch_size)
    val_ds = val_ds.batch(batch_size)

    # Apply prefetching
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.prefetch(tf.data.AUTOTUNE)'''
    
    epochs = 500 #500
    #lr_scheduler = keras.callbacks.LearningRateScheduler(lr_step_decay)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='auto')
    filepath="training_files/best_model_{}.h5".format(val_index[0])
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks = [
        checkpoint,early_stopping
    ]#lr_scheduler early_stopping
    model.compile(
        optimizer=keras.optimizers.experimental.SGD(initial_learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy","TruePositives","TrueNegatives","FalsePositives", "FalseNegatives","Recall","Precision"]
    )
    
    start_time = time.monotonic()
    
    
    hist=model.fit(
        x= trainX_k,
        y = trainY_k,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(valX_k, valY_k)
    )
    
    end_time = time.monotonic()
    print(timedelta(seconds=end_time - start_time))
    
    #### Saves training data of each iteration
    filepath="training_files/model_{}.h5".format(val_index[0])
    model.save_weights(filepath) # verificar load_best_model
    model_json = model.to_json()
    with open("training_files/model_{}.json".format(val_index[0]), "w") as json_file:
        json_file.write(model_json)
    hist_df = pd.DataFrame(hist.history)
    # or save to csv: 
    hist_csv_file = 'training_files/history_{}.csv'.format(val_index[0])
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

2023-12-08 13:05:07.918089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 13:05:07.918131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 13:05:07.919051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 13:05:07.989964: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 13:05:09.421073: W tensorflow/compiler/

Found 1722 files belonging to 2 classes.


2023-12-08 13:05:13.160004: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22240 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:03:00.0, compute capability: 8.6
2023-12-08 13:05:13.362055: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Found 430 files belonging to 2 classes.
TRAIN:  1377 VAL:  345


2023-12-08 13:05:30.458566: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4331667456 exceeds 10% of free system memory.
2023-12-08 13:05:33.643721: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4331667456 exceeds 10% of free system memory.
2023-12-08 13:05:35.848721: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4331667456 exceeds 10% of free system memory.


...

2023-12-08 13:05:42.185519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8801
2023-12-08 13:05:43.400267: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-08 13:05:44.871826: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f49cb701fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-08 13:05:44.871858: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-08 13:05:44.951052: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 [==============================] - ETA: 0s - loss: 1.1127 - accuracy: 0.5882 - true_positives: 419.0000 - true_negatives: 391.0000 - false_positives: 291.0000 - false_negatives: 276.0000 - recall: 0.6029 - precision: 0.5901
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to training_files/best_model_4.h5


/home/igonzalez/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


44/44 [==============================] - 23s 355ms/step - loss: 1.1127 - accuracy: 0.5882 - true_positives: 419.0000 - true_negatives: 391.0000 - false_positives: 291.0000 - false_negatives: 276.0000 - recall: 0.6029 - precision: 0.5901 - val_loss: 0.6692 - val_accuracy: 0.6000 - val_true_positives: 105.0000 - val_true_negatives: 102.0000 - val_false_positives: 77.0000 - val_false_negatives: 61.0000 - val_recall: 0.6325 - val_precision: 0.5769
Epoch 2/500
43/44 [============================>.] - ETA: 0s - loss: 0.8341 - accuracy: 0.6555 - true_positives: 452.0000 - true_negatives: 450.0000 - false_positives: 232.0000 - false_negatives: 242.0000 - recall: 0.6513 - precision: 0.6608
Epoch 2: val_accuracy did not improve from 0.60000
44/44 [==============================] - 9s 199ms/step - loss: 0.8337 - accuracy: 0.6558 - true_positives: 453.0000 - true_negatives: 450.0000 - false_positives: 232.0000 - false_negatives: 242.0000 - recall: 0.6518 - precision: 0.6613 - val_loss: 1.2812 - va

2023-12-08 13:13:53.599801: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4331667456 exceeds 10% of free system memory.
